## Introduction to Colab Momentum
The code below will use the an adjusted momentum strategy on the NASDAQ 100 stocks. The momentum is calculated in the function section as the slope of a linear regression between the starting point and the ending point of each stock. The momentum will be zeroed if the stock has a negative rate of change (ROC), i.e. the last point is lower then the start. One only invests in the highest momentums of a chosen number of stocks (selcperc in Parameters section). As for the momentum strategy, one can adjust how often investments are evaluated, whether one wants to ROC and whether there should be a rebalancing if severeal stocks will be picked from the stock universe.

###Other cool stuff in the Parameters section:
Additionally to the basic momentum strategy there is a simple buy and hold strategy implemented (Pick HSt as strategy). 

One can also evaluate different parameter combinations by setting Heatmap to True. The Heatmap currently allows for two parameters to be evaluated (params_combination & corresponding heatmap_axis) and it will return the given optimization target, e.g. returns in %, Sharpe etc. 

One can also use a particle swarm algorithm (optunity in the Parameters section) to find optimal parameter combinations. Here again the chosen params_combination and the optimization_target are applied.

Last but definitely not least, there is a walk forward analysis that has to be used with the heatmap or the above mentioned algorithm. A walk forward analysis cuts the data into pieces and uses the optimal paramters of part 1 for part 2 and so forth. Thereby the bias of using the optimal parameters for the whole timeline can be evaluated and the strategy can be tested.
Please use the walk forward analysis on a more powerful machinery than colab.

## Jupyter Extensions - convenience as an *extension*

In [1]:
'''
For Jupyter to bring more added value I recommend you to install extensions
pip Commands:

pip install jupyter_nbextensions_configurator jupyter_contrib_nbextensions
jupyter contrib nbextension install --user
jupyter nbextensions_configurator enable --user

My Favorites are:
* Collapsible headings
* Code folding
* Notify
* Hinterland (can be nice, but I think I didn't come to use it correctly)
              
Thanks to https://becominghuman.ai/enhance-your-jupyter-experience-with-these-notebook-widgets-a2717921f678
'''
# stretch the code to 90% of the browser width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Potential error messages & Colab package installation

In [ ]:
#Indexing errors, i.e. ValueError: Length of values (3519) does not match length of index (3529)
    #There are actually 3 data input:
    #1. The folder with all the stocks, 2. the benchmark (I tend to use QQQ), 3. the bond (I used TLT)
    #These errors are in my experience due to one or several of the input csvs do not cover the time span inserted in the "fromdate" / "todate" fields in globalparams, 
    #i.e. the todate goes to 2021 althogh the csvs only cover until mid 2020
#!pip install --user pandas-datareader==0.10
#!pip install --upgrade pandas
!pip install --upgrade pandas-datareader
!pip install backtrader
!pip install quantstats
!pip install pyfolio
!pip install optunity


     |████████████████████████████████| 109 kB 12.0 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0
     |████████████████████████████████| 410 kB 16.7 MB/s 
     |████████████████████████████████| 6.3 MB 28.7 MB/s 
  Created wheel for quantstats: filename=QuantStats-0.0.37-py2.py3-none-any.whl size=38848 sha256=475748bf4b35cc0845fc16dba13800d4bf57ed50eff252755b8500ca5e8b79c1
  Stored in directory: /root/.cache/pip/wheels/bd/c2/f2/cf5b53fff6272dfd581b0e0db59c5e810ae7991ba8723787b9
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=740116739e5ffa8bef0843c8d76ef643bea1d04b253dff149ac3b7feb23a5370
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built quantstats yfinance
  Attempting uninstall: lxml
    Found existing install

## Imports

In [ ]:
import datetime
import glob
import webbrowser
import os.path
import os
import pandas as pd
import backtrader as bt
import backtrader.analyzers as btanal
import pyfolio as pf
import numpy as np
import sys
from scipy.stats import linregress
import quantstats as qs
import seaborn as sns
import optunity.metrics
import math
import plotly.graph_objects as go
import multiprocess #can pickle when multiprocessing can't
 
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Rectangle
 
%matplotlib inline
import matplotlib.pyplot as plt
 
import warnings 
warnings.simplefilter(action='ignore', category=Warning)
warnings.simplefilter(action='ignore', category=FutureWarning)
 
#Imports for walkforward analysis
from sklearn.model_selection import TimeSeriesSplit
from sklearn.utils import indexable
from sklearn.utils.validation import _num_samples
import backtrader.indicators as btind
import datetime as dt
import pandas as pd
import pandas_datareader as web
from pandas import Series, DataFrame
import random
import optunity.metrics
import backtrader.analyzers as btanal

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


In [ ]:
from google.colab import drive
!fusermount -u drive
!google-drive-ocamlfuse drive
drive.mount('/gdrive', force_remount=True)

fusermount: failed to unmount /content/drive: No such file or directory
/bin/bash: google-drive-ocamlfuse: command not found
Mounted at /gdrive


## Parameters

In [ ]:
globalparams = dict(path='/gdrive/MyDrive/Stock Trading/NASDAQ2021/',      # location of your csv files, downloaded from Yahoo.
                    fromdate=datetime.datetime(2020, 6, 10),
                    todate=datetime.datetime(2021, 10, 2),
                    strategy="St",                          # HSt = Buy and Hold, St = Standard Strategy for non-survivorship free bias momentum
                    update_csv=True,                        # If True, the csv files will be updated with the timeframe given in fromdate/todate
                    
                    # Momentum Strategy specifics(= St)        
                    mperiod=200,           # lookback period for momentum - default 200 periods
                    selcperc=2,            # number of stocks to select from the universe
                    time_interval="week", # if it is not "week" then it is "month"
                    time_between=1,        # time between time interval, e.g. 1 means weekly, 2 means every 2 weeks etc. same for months
                    
                    roc = True,        # Whether the rate of change should be taken into consideration
                    rocperiod=200,     # rate of change to make sure stock is increasing
                    
                    rebal=True,            # apply proper rebalancing when stocks are sold than have a share above the equal weights
 
                    #General optimization parameters
                    params_combination= "23",               # 1: # of stocks picked from universe, 2: momentum period, 3: roc period
                    optimization_target="SQN",          # Relevant for heatmap & optunity. 
                                                            # Potential Inputs: returns, Sharpe, Sortino, SQN (needs more than 30 trades!), VWR (variablility weighted returns)
                    heatmap_yaxisr=np.arange(1, 7, 3),      # y axis range of parameters for heatmap, last position is the step in between
                    heatmap_xaxisr=np.arange(100, 210, 50), # x axis (mperiod) range of parameters for heatmap
                    heatmap_zaxisr=np.arange(100, 210, 50), # z axis (ROC) range of parameters for heatmap                    
                    
                    #Heatmap section
                    heatmap=False,  # Does not work with Hold and Buy, Multiprocessing only here implemented (works in normal and walkforward runs)                  
                    
                    #optimization with optunity (various algorithms e.g. Partical Swarm Optimization)
                    optunity=False, # Either optunity or heatmap, Multiprocessing not implemented
                    num_eval=1,    # how many evaluations optunity will try // also for Walk Forward Analysis                    
                    
                    #Walk-forward-analysis, either optunity or heatmap must be switched on
                    walk=False,
                    fixed_length=False, # by setting it False the training data will will grow over time, otherwise it will keep the size given under train_splits
                    n_splits=3,         # how many chunks the data should have
                    
                    cash=1000000,               # amount of initial captal      
                    commission=0.005,           # broker fee/ask/bid spread
                    reserve=0.05,               # % of cash that won't be spent due to otherwise canceled orders
                    printlog=True,
                    ranking_timeanalysis=True, # excel file with rankings of all stocks over time, only works in "normal" run (no heatmap/Walk-forward/optunity), 
                                                # if it doesn't work, shorten the time period in fromdate
                    )

## Downloading data from Yahoo
Enter symbol, from , to, path

In [ ]:
if globalparams["update_csv"]:
    from csv import reader
    # open file in read mode
    from1 = globalparams["fromdate"]
    to = globalparams["todate"]
    #Download all 
    with open(r'/gdrive/MyDrive/Stock Trading/Others/NASDAQ2021.csv', 'r') as read_obj:
        # pass the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
        # Iterate over each row in the csv using reader object
        for row in csv_reader:
            # row variable is a list that represents a row in csv
            string = ''
            path = r'/gdrive/MyDrive/Stock Trading/NASDAQ2021/'
            hist = web.DataReader(string.join(row), 'yahoo', from1, to)
            #hist = yf.HistoricalPrices(string.join(row),  from1, to, frequency='1d')
            hist.to_csv(path + string.join(row) + '.csv')
    #Download Benchmark
    row = "QQQ"
    string = ''
    path = '/gdrive/MyDrive/Stock Trading/Others/'
    hist = web.DataReader(string.join(row), 'yahoo', from1, to)
    #hist = yf.HistoricalPrices(string.join(row),  from1, to, frequency='1d')
    hist.to_csv(path + string.join(row) + '.csv')

## Function section
Momentum (with and without volatility correction) [thanks to Teddy Koker https://teddykoker.com/2019/05/momentum-strategy-from-stocks-on-the-move-in-python/ ]  and heatmap for analysis of variable combinations and [thanks to samwindham1 https://github.com/samwindham1/algo-trader/blob/master/backtest/util/analyzers.py ]

In [ ]:
#Momentum + ROC
class MomentumEnhanced(bt.Indicator):
    lines = ('trend',)
    params = (('period', 200), ('rperiod', 200))
    
    def __init__(self):
        self.addminperiod(self.params.period)
        if globalparams["roc"]:
            self.roc = bt.ind.ROC(self.data, period=self.p.rperiod)
                
    def next(self):              
        #Momentum
        returns = np.log(self.data.get(size=self.p.period))
        x = np.arange(len(returns))
        slope, _, rvalue, _, _ = linregress(x, returns)
        momentum_annualized = (1 + slope) ** 252
        
        #ROC
        rocround = math.ceil(self.roc[0]) if globalparams["roc"] else 1
        roc_final = min(1, max(0, rocround)) if globalparams["roc"] else 1

        #final line formula
        self.lines.trend[0] = momentum_annualized * roc_final

# Great Heatmap to analyse different combinations of parameters
def my_heatmap1(data):
    data = np.array(data)
    xs = np.unique(data[:, 1].astype(int))
    ys = np.unique(data[:, 0].astype(int))
    vals = data[:, 2].reshape(len(ys), len(xs))
    
    min_val_ndx = np.unravel_index(np.argmin(vals, axis=None), vals.shape)
    max_val_ndx = np.unravel_index(np.argmax(vals, axis=None), vals.shape)
    
    cmap = LinearSegmentedColormap.from_list('', ['red', 'orange', 'yellow', 'chartreuse', 'limegreen'])
    ax = sns.heatmap(vals, xticklabels=xs, yticklabels=ys, cmap='viridis', annot=True, fmt='.2f')

    ax.add_patch(Rectangle(min_val_ndx[::-1], 1, 1, fill=False, edgecolor='blue', lw=3, clip_on=False))
    ax.add_patch(Rectangle(max_val_ndx[::-1], 1, 1, fill=False, edgecolor='blue', lw=3, clip_on=False))
    print("Average of all iterations", [sum(i)/len(i) for i in zip(*data)][2])
    #plt.tight_layout()
    
    plt.show()

def plotly_4d(data):
    df1 = pd.DataFrame.from_records(data, columns=['selcperc', 'mperiod', 'rocperiod', 'value'])
    df = df1[~(df1['selcperc'].isin([2]))]
    df= df.pivot_table(index=['mperiod'], columns=['rocperiod'], values='value').reset_index()
    del df['mperiod']
    df.drop(df.index[0])
    print(df)
    fig = go.Figure(data=[go.Surface(z=df.values)])
    fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                      highlightcolor="limegreen", project_z=True))
    fig.update_layout(title_text="Analysis of Strategy", autosize=False, width=700, height=700,
                  margin=dict(l=65, r=50, b=65, t=90))
    fig.show() 
  
    
class Sortino(bt.Analyzer):
    params = (('timeframe', bt.TimeFrame.Days),('compression', 1),('riskfreerate', 0.00),('factor', None),('convertrate', True),('annualize', True),)

    RATEFACTORS = {
        bt.TimeFrame.Days: 252,
        bt.TimeFrame.Weeks: 52,
        bt.TimeFrame.Months: 12,
        bt.TimeFrame.Years: 1,
    }

    def __init__(self):
        super(Sortino, self).__init__()
        self.ret = btanal.TimeReturn(
            timeframe=self.p.timeframe,
            compression=self.p.compression)
        self.ratio = 0.0

    def stop(self):
        returns = list(self.ret.get_analysis().values())

        rate = self.p.riskfreerate

        factor = None
        if self.p.timeframe in self.RATEFACTORS:
            # Get the conversion factor from the default table
            factor = self.RATEFACTORS[self.p.timeframe]

        if factor is not None:
            # A factor was found
            if self.p.convertrate:
                # Standard: downgrade annual returns to timeframe factor
                rate = pow(1.0 + rate, 1.0 / factor) - 1.0
            else:
                # Else upgrade returns to yearly returns
                returns = [pow(1.0 + x, factor) - 1.0 for x in returns]

        if len(returns):
            # Sortino Ratio = (R - T) / TDD
            #   R = Avg Returns
            #   T = Target (risk-free rate)
            #   TDD = Downside Risk
            ret_free_avg = np.mean(returns) - rate
            tdd = math.sqrt(np.mean([min(0, r - rate)**2 for r in returns]))

            try:
                ratio = ret_free_avg / tdd

                if factor is not None and \
                        self.p.convertrate and self.p.annualize:

                    ratio = math.sqrt(factor) * ratio
            except (ValueError, TypeError, ZeroDivisionError):
                ratio = None

        else:
            # no returns
            ratio = None

        self.ratio = ratio

    def get_analysis(self):
        return dict(sortino=self.ratio)

# Picking optimization target and subsequent analyser functions for optimization
def opta(i):
    switcher={
            "VWR":['cerebro.addanalyzer(btanal.VWR, _name="VWR")','results1[0].analyzers.VWR.get_analysis()["vwr"]'],
            "Sortino":['cerebro.addanalyzer(Sortino, _name="sortino")','results1[0].analyzers.sortino.get_analysis()["sortino"]'],
            "returns":['1==1','cerebro.broker.getvalue()'], #TODO must be tested separately
            "Sharpe":['cerebro.addanalyzer(btanal.SharpeRatio, _name="sharpe")','results1[0].analyzers.sharpe.get_analysis()["sharperatio"]'],
            "SQN":['cerebro.addanalyzer(btanal.SQN, _name="sqn")','results1[0].analyzers.sqn.get_analysis()["sqn"]'],
         }
    return switcher.get(i,"Optimization Target not available")

def params_comb(i): 
    switcher={
            "12":[['globalparams["heatmap_yaxisr"]','globalparams["heatmap_xaxisr"]','[globalparams["rocperiod"],globalparams["rocperiod"]*1.0001]'],['results_list.append([i, j, k, PnL])','2','1','2']], #p1, p2, p3 in optunity maximize
            "13":[['globalparams["heatmap_yaxisr"]','[globalparams["mperiod"],globalparams["mperiod"]*1.0001]','globalparams["heatmap_zaxisr"]'],['results_list.append([i, k, j, PnL])','1','2','1']],
            "23":[['[globalparams["selcperc"],globalparams["selcperc"]*1.0001]','globalparams["heatmap_xaxisr"]','globalparams["heatmap_zaxisr"]'],['results_list.append([j, k, i, PnL])','0','0','1']],
            "123":[['globalparams["heatmap_yaxisr"]','globalparams["heatmap_xaxisr"]','globalparams["heatmap_zaxisr"]'],['results_list.append([i, j, k, PnL])','0','1','2']],
         }
    return switcher.get(i,"Optimization Target not available")

def params_comb_optunity(i): 
    switcher={
            "12":[['[globalparams["heatmap_yaxisr"][0],globalparams["heatmap_yaxisr"][-1]]','[globalparams["heatmap_xaxisr"][0],globalparams["heatmap_xaxisr"][-1]]','[globalparams["rocperiod"],globalparams["rocperiod"]*1.0001]'],['results_list.append([i, j, k, PnL])','0','1','2']], #p1, p2, p3 in optunity maximize
            "13":[['[globalparams["heatmap_yaxisr"][0],globalparams["heatmap_yaxisr"][-1]]','[globalparams["mperiod"],globalparams["mperiod"]*1.0001]','[globalparams["heatmap_zaxisr"][0],globalparams["heatmap_zaxisr"][-1]]'],['results_list.append([i, k, j, PnL])','0','2','1']],
            "23":[['[globalparams["selcperc"],globalparams["selcperc"]*1.0001]','[globalparams["heatmap_xaxisr"][0],globalparams["heatmap_xaxisr"][-1]]','[globalparams["heatmap_zaxisr"][0],globalparams["heatmap_zaxisr"][-1]]'],['results_list.append([j, k, i, PnL])','2','0','1']],
            "123":[['[globalparams["heatmap_yaxisr"][0],globalparams["heatmap_yaxisr"][-1]]','[globalparams["heatmap_xaxisr"][0],globalparams["heatmap_xaxisr"][-1]]','[globalparams["heatmap_zaxisr"][0],globalparams["heatmap_zaxisr"][-1]]'],['results_list.append([i, j, k, PnL])','0','1','2']],
         }
    return switcher.get(i,"Optimization Target not available")

## Momentum Strategy
Originally it is a combination of the momentum strategy example given on the official website:
https://www.backtrader.com/blog/2019-05-20-momentum-strategy/momentum-strategy/
and  https://teddykoker.com/2019/05/momentum-strategy-from-stocks-on-the-move-in-python/

In [ ]:
class St(bt.Strategy):
    params = dict(
        selcperc=globalparams["selcperc"],  # percentage of stocks to select from the universe
        mperiod=globalparams["mperiod"],  # lookback period for momentum - default 90 periods
        rocperiod=globalparams["rocperiod"],   # rate of change to make sure stock is increasing
        momentumplus=MomentumEnhanced, # choose which momentum function is used
        reserve=globalparams["reserve"],  # 5% reserve capital
        monthdays=[1],
        monthcarry=True,
        weekdays=[5],
        weekcarry=True,
        printlog=globalparams["printlog"],
        WFA=False,          #Switch for the last "putting the fragments together run" of the Walk Forward Analysis
        start_dates=None,  # Starting days for trading periods (a list)
        end_dates=None,
        
    )

    def __init__(self):
        self.bench = self.data0
        #self.bond = self.data1
        self.stocks = self.datas[1:]
        # calculate 1st the amount of stocks that will be selected

        # counter for time in between
        self.timer_i = 0
        self.posdata =[]
        
        # This is the set up of the timer that makes the strategy being executed at the given time
        if globalparams["time_interval"] == "week":
            self.add_timer(
                when=bt.timer.SESSION_START,
                weekdays=self.p.weekdays,
                weekcarry=self.p.weekcarry,
            )  
        else:
            self.add_timer(
                when=bt.timer.SESSION_START,
                monthdays=self.p.monthdays,
                monthcarry=self.p.monthcarry,
            )

        # preparation for Walk Forward Analysis
        if self.p.WFA == True:
            self.mperiod = dict()
            self.vperiod = dict()
            self.ms = dict()
            self.selnum = dict()

            self.date_combos = [c for c in zip(self.p.start_dates, self.p.end_dates)]

            # Additional indexing, allowing for differing start/end dates
            for sd, ed, f, s, t in zip(self.p.start_dates, self.p.end_dates, self.p.mperiod, self.p.rocperiod, self.p.selcperc):
                self.selnum[(sd, ed)] = int(t) # of stocks bought
                
                # returns, volatilities and momentums
                self.ms[(sd, ed)] = [self.p.momentumplus(d, period=int(f), rperiod=int(s)) for d in self.stocks] #toDo how to implement it with two parameters
        
        #Non walk forward analysis
        else:
            self.selnum = int(self.p.selcperc)
            # allocation perc per stock
            ms = [self.p.momentumplus(d,period=int(self.p.mperiod), rperiod=int(self.p.rocperiod)) for d in self.stocks]

            # simple rank formula: (momentum * net payout) / volatility
            # the highest ranked: low vol, large momentum, large payout
            self.ranks = {d: m for d, m in zip(self.stocks, ms)}
            
                
    def log(self, arg):
        if self.p.printlog:
            print('{} {}'.format(self.datetime.date(), arg))
        
    # This section is for logging of orders in greater detail to figure out whether the strategy is actually having no problem with orders
    def notify_order(self, order):
        if order.status in [order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        if order.status in [order.Submitted]:
            if order.isbuy():
            
                dt, dn = self.datetime.date(), order.data._name
                self.log('Buy {} {} {} Price {:.2f} Value {:.2f} Size {} Cash {:.2f}'.format(
                        order.getstatusname(), dt, dn, order.created.price, order.created.size * order.created.price , order.created.size, self.broker.getcash()))
            if order.issell():
                dt, dn = self.datetime.date(), order.data._name
                self.log('Sell {} {} {} Price {:.2f} Value {:.2f} Size {}'.format(
                        order.getstatusname(), dt, dn, order.created.price, order.created.size * order.created.price, order.created.size))

            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                dt, dn = self.datetime.date(), order.data._name
                self.log('Buy {} {} Price {:.2f} Value {:.2f} Size {}'.format(
                    dt, dn, order.executed.price, order.executed.value, order.executed.size))
                
            if order.issell():# Sell
                dt, dn = self.datetime.date(), order.data._name
                self.log('Sell {} {} Price {:.2f} Value {:.2f} Size {}'.format(
                    dt, dn, order.executed.price, order.executed.price*order.executed.size, order.executed.size))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
           
    
    # This is the function using the timer to execute the rebalance 
    def notify_timer(self, timer, when, *args, **kwargs):
        #print('strategy notify_timer with tid {}, when {} _getminperstatus {}'.
        #      format(timer.p.tid, when, int(self._getminperstatus())))
        if self._getminperstatus() < 0:
            self.timer_i += 1
            if self.timer_i == globalparams["time_between"]:
                self.timer_i = 0
                self.rebalance()
  
    def next(self):
        pass # must be filled with a pass

    
    # Actual order giving by a ranking takes place here
    def rebalance(self):
        
        # Walkforward Analysis Section (no triple momentum possible)
        if self.p.WFA == True:
            # Determine which set of moving averages to use
            curdate = self.datetime.date(0)
            dtidx = None  # Will be index
            # Determine which period (if any) we are in
            for sd, ed in self.date_combos:
                if sd <= curdate and curdate <= ed:
                    dtidx = (sd, ed)
                    self.ranks = {d: m for d, m in zip(self.stocks, self.ms[dtidx])}
                    
                    # sort data and current rank
                    ranks = sorted(
                        self.ranks.items(),  # get the (d, rank), pair
                        key=lambda x: x[1][0],  # use rank (elem 1) and current time "0"
                        reverse=True,  # highest ranked 1st ... please
                    )

                    # put top ranked in dict with data as key to test for presence
                    rtop = dict(ranks[:self.selnum[dtidx]])

                    # For logging purposes of stocks leaving the portfolio
                    rbot = dict(ranks[self.selnum[dtidx]:])

                    # prepare quick lookup list of stocks currently holding a position
                    self.posdata = [d for d, pos in self.getpositions().items() if pos]
                    
            if dtidx is None:  # Not in any window
                pass # Don't engage in trades
            else:
                self.momentum_core(self.posdata, rtop, rbot, dtidx)             

        # Non walk forward section
        else:
            # sort data and current rank
            ranks = sorted(
                self.ranks.items(),  # get the (d, rank), pair
                key=lambda x: x[1][0],  # use rank (elem 1) and current time "0"
                reverse=True,  # highest ranked 1st ... please
            )
            
            # put top ranked in dict with data as key to test for presence
            rtop = dict(ranks[:self.selnum])

            # For logging purposes of stocks leaving the portfolio
            rbot = dict(ranks[self.selnum:])
            
            # prepare quick lookup list of stocks currently holding a position
            self.posdata = [d for d, pos in self.getpositions().items() if pos]
                
            self.momentum_core(self.posdata, rtop, rbot, 0)
            
    def momentum_core(self, posdata, rtop, rbot, dtidx):
        # remove those no longer top ranked
        # do this first to issue sell orders and free cash
        sell = False
        for d in (d for d in posdata if d not in rtop):
            self.log('Leave {} - Rank {:.2f}'.format(d._name, rbot[d][0]))
            self.order_target_percent(d, target=0.0)
            if self.p.WFA == True:
                if (0.95 / self.selnum[dtidx]) < self.broker.getposition(d).size * self.broker.getposition(d).adjbase / self.broker.get_value():
                    sell = globalparams["rebal"]
            else:
                if (0.95 / self.selnum) < self.broker.getposition(d).size * self.broker.getposition(d).adjbase / self.broker.get_value():
                    sell = globalparams["rebal"]

        sum_stocks = 0
        # rebalance those already top ranked and still there
        for d in (d for d in posdata if d in rtop):
            if self.p.WFA == True:
                if sell and ((0.95 / self.selnum[dtidx]) < self.broker.getposition(d).size * self.broker.getposition(d).adjbase / self.broker.get_value()):
                    self.log('Rebal {} - Rank {:.2f}'.format(d._name, rtop[d][0]))
                    self.order_target_percent(d, target=0.95 * rtop[d][0]/ (0.95 / self.selnum[dtidx]))
            else:
                if sell and ((0.95 / self.selnum) < self.broker.getposition(d).size * self.broker.getposition(d).adjbase / self.broker.get_value()):
                    self.log('Rebal {} - Rank {:.2f}'.format(d._name, rtop[d][0]))
                    self.order_target_percent(d, target=0.95 * rtop[d][0]/(0.95 / self.selnum))
                    
            #Rebalances are just pricy operations which are needed for the buying part ... since it was based on equal weights...
            del rtop[d]  # remove it, to simplify next iteration
            sum_stocks += self.broker.getposition(d).size * self.broker.getposition(d).adjbase # Calulate the value of the stocks owned


        if len(rtop):
            sizer_perc = (0.95 - sum_stocks / self.broker.get_value()) / len(rtop) # Have a equal sizer for the cash (you want to invest in)
            sum_stocks = 0

        # issue a target order for the newly top ranked stocks
        # do this last, as this will generate buy orders consuming cash
        for d in rtop:
            self.log('Enter {} - Rank {:.2f}'.format(d._name, rtop[d][0]))
            self.order_target_percent(d, target=sizer_perc)
           
    def stop(self):
        myrankings=pd.DataFrame()
        if globalparams["ranking_timeanalysis"]:
            t = 1
            for k,v in self.ranks.items():
                if t == 1:
                    time = [k.num2date(x) for x in (k.datetime).get(ago=0, size=len(k))]
                    myrankings = pd.DataFrame(columns=[k._name], index=time)
                    t +=1
                else:
                    myrankings[k._name] = 1
                list = []
                for i in v.get(size=len(v)):
                    list.append(i)
                myrankings[k._name] = list
            myrankings.to_excel(r'/gdrive/MyDrive/Stock Trading/Others/output.xlsx')
        pnl = round(self.broker.getvalue() - globalparams["cash"],2)
        self.log('Final PnL: {}'.format(
            pnl))

## Walk forward analysis
Thanks to https://ntguardian.wordpress.com/2017/06/19/walk-forward-analysis-demonstration-backtrader/

In [ ]:
def walk_forward_analysis(datafeeds):
    # display dataframes more orderly
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 10)
    pd.set_option('display.width', 1000)

    globalparameters = dict(strategy=globalparams['strategy'],            # if a different strategy is used
                        n_splits=globalparams["n_splits"],                # how many chunks the data should have
                        fixed_length=globalparams["fixed_length"],         # by setting it False the training data will will grow over time, otherwise it will keep the size given under train_splits
                        train_splits=1,             # how many splits should be used to train the model (be aware of these two variables may not work with your strategy, i.e. SMA100 but two training splits are just 110 days or less than 100 days)
                        test_splits=1,              # how many splits should the data be tested on?
                        cash=globalparams["cash"],
                        commission=globalparams["commission"],
                        num_evals=globalparams["num_eval"],               # how often should the optimizer try to optimize
                        )


    class TimeSeriesSplitImproved(TimeSeriesSplit):

        def split(self, X, y=None, groups=None, fixed_length=False,
                  train_splits=1, test_splits=1):

            X, y, groups = indexable(X, y, groups)
            n_samples = _num_samples(X)
            n_splits = self.n_splits
            n_folds = n_splits + 1
            train_splits, test_splits = int(train_splits), int(test_splits)
            if n_folds > n_samples:
                raise ValueError(
                    ("Cannot have number of folds ={0} greater"
                     " than the number of samples: {1}.").format(n_folds,
                                                                 n_samples))
            if ((n_folds - train_splits - test_splits) == 0 and test_splits > 0):
                raise ValueError(
                    ("Both train_splits and test_splits must be positive"
                     " integers."))
            indices = np.arange(n_samples)
            split_size = (n_samples // n_folds)
            test_size = split_size * test_splits
            train_size = split_size * train_splits
            test_starts = range(train_size + n_samples % n_folds,
                                n_samples - (test_size - split_size),
                                split_size)

            if fixed_length:
                for i, test_start in zip(range(len(test_starts)),
                                         test_starts):
                    rem = 0
                    if i == 0:
                        rem = n_samples % n_folds
                    yield (indices[(test_start - train_size - rem):test_start],
                           indices[test_start:test_start + test_size])
            else:
                for test_start in test_starts:
                    yield (indices[:test_start],
                           indices[test_start:test_start + test_size])

    class AcctStats(bt.Analyzer):
        """A simple analyzer that gets the gain in the value of the account; should be self-explanatory"""

        def __init__(self):
            self.start_val = self.strategy.broker.get_value()
            self.end_val = None

        def stop(self):
            self.end_val = self.strategy.broker.get_value()

        def get_analysis(self):
            return {"start": self.start_val, "end": self.end_val,
                    "growth": self.end_val - self.start_val, "return": self.end_val / self.start_val}

    for df in datafeeds.values():
        df["OpenInterest"] = 0  # PandasData reader expects an OpenInterest column;

    tscv = TimeSeriesSplitImproved(globalparameters['n_splits'])
    split = tscv.split(datafeeds[list(datafeeds.keys())[2]], fixed_length=globalparameters['fixed_length'], train_splits=globalparameters['train_splits'], test_splits=globalparameters['test_splits'])
    walk_forward_results = list()
    
    heatmap_data = []
    # Be prepared: this will take a while
    i = 0
    for train, test in split:
        i += 1
        # TRAINING
        # Optimize with optunity
        def runstrat(p1=globalparams["selcperc"], p2=globalparams["mperiod"], p3=globalparams["rocperiod"], WFA=False):
            cerebro = bt.Cerebro(maxcpus=0,stdstats=False, runonce=False)
            cerebro.addstrategy(eval(globalparameters['strategy']), selcperc=p1, mperiod=p2, rocperiod=p3, printlog=False) #change the parameters that should be depicted in heatmap
            cerebro.broker.setcash(globalparams["cash"])
            cerebro.broker.setcommission(commission=globalparams["commission"])
            eval(opta(globalparams["optimization_target"])[0])
            for s, df in datafeeds.items():
                data = bt.feeds.PandasData(dataname=df.iloc[train], name=s)  # Add a subset of data
                cerebro.adddata(data)     
            cerebro.broker.set_coc(True)
            results1 = cerebro.run()
            print(round(p1,0), round(p2,0), round(p3,0), round(eval(opta(globalparams["optimization_target"])[1]),2), round(cerebro.broker.getvalue(),0))
            if globalparams["heatmap"]: 
                return [round(p1,0), round(p2,0), round(p3,0), round(eval(opta(globalparams["optimization_target"])[1]),2), round(cerebro.broker.getvalue()/globalparams["cash"],2)]
            if globalparams["optunity"]:
                return round(eval(opta(globalparams["optimization_target"])[1]),2)

        #Either use Optunity for quick search of optimum
        if globalparams["optunity"]:
            opt = optunity.maximize(runstrat, num_evals=globalparams["num_eval"], pmap=optunity.pmap, p1=eval(params_comb_optunity(globalparams["params_combination"])[0][0]), 
                                    p2=eval(params_comb_optunity(globalparams["params_combination"])[0][1]), 
                                    p3=eval(params_comb_optunity(globalparams["params_combination"])[0][2]))
            optimal_pars, details, _ = opt
            optimal_pars['var1'] = int(optimal_pars['p1'])
            optimal_pars['var2'] = int(optimal_pars['p2'])
            optimal_pars['var3'] = int(optimal_pars['p3'])
        
        # or heatmap showing how variables i = (y axis/ p1) and j = (x axis /p2) impact strategy outcome
        if globalparams["heatmap"]:     
            pool = multiprocess.Pool(processes=7, maxtasksperchild=1)  
            noduplicates = []
            runt = 1
            for i in eval(params_comb(globalparams["params_combination"])[0][0]):
                i = int(i)
                for j in eval(params_comb(globalparams["params_combination"])[0][1]):
                    j = int(j)
                    for k in eval(params_comb(globalparams["params_combination"])[0][2]):  
                        k = int(k)
                        if runt == 1:
                            noduplicates.append([i, j, k])
                        else:
                            if [i, j, k] in noduplicates:
                                continue
                            else:
                                noduplicates.append([i, j, k])                            
                        runt += 1
                        
            def Convert(lst):
                lst2 = [["p1","p2","p3"] for x in range(len(lst))]
                res_dct = []
                for x in range(len(lst)):
                    res_dct.append(dict(zip(lst2[x],lst[x])))
                return res_dct
            
            Convert(noduplicates)

            results_list = pool.starmap_async(runstrat, noduplicates,chunksize=1)
            pool.close()            
            pool.join()
            
            #sort list since multiprocessing messes up the order
            results_list = sorted(results_list.get(), key = lambda x:(x[0], x[1], x[2]))
            
            if globalparams["heatmap"]:
                optimal_pars = {}
                #print(results_list)
                #best return from the heatmap loop
                #print(max(results_list, key=lambda x: x[3]), max(results_list, key=lambda x: x[3])[0], datafeeds[list(datafeeds.keys())[2]].iloc[test[0]].name)
                optimal_pars['var1'] = max(results_list, key=lambda x: x[3])[0]
                optimal_pars['var2'] = max(results_list, key=lambda x: x[3])[1]
                optimal_pars['var3'] = max(results_list, key=lambda x: x[3])[2]
                print("Optimal vars 1/2/3 ",optimal_pars['var1'], optimal_pars['var2'], optimal_pars['var3'])
            
            if len(globalparams["params_combination"]) == 2:
                for i in range(len(results_list)):
                    results_list[i].pop(eval(params_comb(globalparams["params_combination"])[1][1]))
                my_heatmap1(results_list)
                for i in range(len(results_list)):
                    results_list[i].pop(2)
                my_heatmap1(results_list)
            else:
                plotly_4d(results_list)
            
        
        results_list = []
        
        # TESTING
        tester = bt.Cerebro(stdstats=False, maxcpus=None, runonce=False)
        tester.broker.set_cash(globalparameters['cash'])
        tester.broker.set_coc(True)
        tester.broker.setcommission(globalparameters['commission'])
        tester.addanalyzer(AcctStats)
        tester.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)

        tester.addstrategy(eval(globalparameters['strategy']), selcperc=optimal_pars['var1'], mperiod=optimal_pars['var2'], rocperiod=optimal_pars['var3'], 
                           WFA=False, printlog=False)  # Test with optimal combination toDO like above int vs float
        for s, df in datafeeds.items():
            data = bt.feeds.PandasData(dataname=df.iloc[test], name=s)  # Add a subset of data
            tester.adddata(data)

        res = tester.run()
        res_dict = res[0].analyzers.acctstats.get_analysis()
        res_dict["var1"] = optimal_pars['var1']
        res_dict["var2"] = optimal_pars['var2']
        res_dict["var3"] = optimal_pars['var3']
        res_dict["sharpe"] = res[0].analyzers.sharperatio.get_analysis()['sharperatio']
        res_dict["start_date"] = datafeeds[list(datafeeds.keys())[2]].iloc[test[0]].name
        res_dict["end_date"] = datafeeds[list(datafeeds.keys())[2]].iloc[test[-1]].name
        walk_forward_results.append(res_dict)
    
    
    wfdf = DataFrame(walk_forward_results)
    print(wfdf.loc[:, wfdf.columns != 'start'])

    cerebro_wf = bt.Cerebro(stdstats=False, maxcpus=None, runonce=False)

    for s, df in datafeeds.items():
        data = bt.feeds.PandasData(dataname=df, name=s, fromdate=wfdf.iloc[0, wfdf.columns.get_loc('start_date')], todate=wfdf.iloc[-1, wfdf.columns.get_loc('end_date')])
        cerebro_wf.adddata(data)  # Give the data to cerebro

    cerebro_wf.broker.setcash(globalparameters['cash'])
    cerebro_wf.broker.setcommission(globalparameters['commission'])
    cerebro_wf.broker.set_coc(True)

    cerebro_wf.addstrategy(eval(globalparams['strategy']),
                           # Give the results of the above optimization to SMACWalkForward (NOT OPTIONAL)
                           selcperc=[f for f in wfdf.var1],
                           mperiod=[s for s in wfdf.var2],
                           rocperiod=[t for t in wfdf.var3],
                           start_dates=[sd.date() for sd in wfdf.start_date],
                           end_dates=[ed.date() for ed in wfdf.end_date],
                           WFA=True,
                           )

    cerebro_wf.addobservermulti(bt.observers.BuySell)  # Plots up/down arrows
    cerebro_wf.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
    cerebro_wf.addanalyzer(btanal.PyFolio)

    results = cerebro_wf.run()
    print(f"Sharpe: {results[0].analyzers.sharperatio.get_analysis()['sharperatio']:.3f}")
    
    # Quantstats thanks to https://algotrading101.com/learn/backtrader-for-backtesting/
    returns, positions, transactions, gross_lev = results[0].analyzers.pyfolio.get_pf_items()
    returns.index = returns.index.tz_convert(None)
    qs.reports.html(returns, output='stats.html', title='Walkforward '+ globalparams["strategy"])
    webbrowser.open('stats.html')

## Exection section

In [ ]:
def run(args=None):
    #optreturn=False otherwise the heatmap doesn't work
    cerebro = bt.Cerebro(maxcpus=1, preload=False, optdatas=False, optreturn=False, stdstats=True) 
    
    #plotsize
    plt.rcParams['figure.figsize'] = [7, 7]
    plt.rcParams['figure.dpi'] = 200
        
    # <<<Data loading section>>>
    # Parse from/to-date
    fromdate = globalparams["fromdate"]
    todate = globalparams["todate"]
    datafeeds = {}
    
    # Add SPY/QQQ as "Benchmark"
    df0 = pd.read_csv('/gdrive/MyDrive/Stock Trading/Others/QQQ.csv', index_col=0, parse_dates=True)
    benchdata = bt.feeds.PandasData(dataname=df0,name="QQQ",fromdate=fromdate, todate=todate,  plot=False)
    cerebro.adddata(benchdata)
    df0 = df0.loc[fromdate : todate]
    datafeeds.update({"QQQ": df0})
 
    # add all the data files available in the directory datadir
    for fname in glob.glob(os.path.join(globalparams["path"], '*')):
        df = pd.read_csv(fname, index_col=0, parse_dates=True)
        # Add dataframes to a dictionary for walkforward analysis
        if len(df)>210:
            cerebro.adddata(bt.feeds.PandasData(dataname=df,name=os.path.basename(fname).replace(".csv", ""),fromdate=fromdate, todate=todate, plot=False))
        df = df.loc[fromdate : todate]
        datafeeds.update({os.path.basename(fname).replace(".csv", ""): df})
            
    results_list = []
    # <<<Cerebro loading section>>>
    if globalparams["walk"]:
        walk_forward_analysis(datafeeds)
    else:
        ready_list = []
        def dummy_func(index):
            global ready_list
            ready_list.append(index)
            
        # Optimization by Optunity/Heatmap (must be stated outside of other classes/functions)
        def runstrat(p1=globalparams["selcperc"], p2=globalparams["mperiod"], p3=globalparams["rocperiod"]):
            cerebro = bt.Cerebro(maxcpus=0,stdstats=True)
            cerebro.addstrategy(St, selcperc=p1, mperiod=p2, rocperiod=p3, printlog=False) #change the parameters that should be depicted in heatmap
            cerebro.broker.setcash(globalparams["cash"])
            cerebro.broker.setcommission(commission=globalparams["commission"])
            eval(opta(globalparams["optimization_target"])[0])
            cerebro.adddata(benchdata) 
            for fname in glob.glob(os.path.join(globalparams["path"], '*')):
                df = pd.read_csv(fname, index_col=0, parse_dates=True)
                if len(df)>200:
                    cerebro.adddata(bt.feeds.PandasData(dataname=df,name=os.path.basename(fname).replace(".csv", ""),fromdate=fromdate, todate=todate, plot=False))        
 
            cerebro.broker.set_coc(True)
            results1 = cerebro.run()
            print(round(p1,0), round(p2,0), round(p3,0), round(eval(opta(globalparams["optimization_target"])[1]),2), round(cerebro.broker.getvalue()/globalparams["cash"],2))
 
            #the return is as far as I can see just legacy since the line above will be used
            if globalparams["heatmap"]: 
                return [round(p1,0), round(p2,0), round(p3,0), round(eval(opta(globalparams["optimization_target"])[1]),1), round(cerebro.broker.getvalue()/globalparams["cash"],1)]
            if globalparams["optunity"]:
                return round(eval(opta(globalparams["optimization_target"])[1]),2)
            
        #Either use Optunity for quick search of optimum
        if globalparams["optunity"]:
            opt = optunity.maximize(runstrat, num_evals=globalparams["num_eval"], pmap=optunity.pmap, p1=eval(params_comb_optunity(globalparams["params_combination"])[0][0]), 
                                    p2=eval(params_comb_optunity(globalparams["params_combination"])[0][1]), 
                                    p3=eval(params_comb_optunity(globalparams["params_combination"])[0][2]))
            optimal_pars, details, _ = opt
            print('Optimal Parameters:')
            print('selcperc = %.4f' % optimal_pars['p1'])
            print('mperiod = %.4f' % optimal_pars['p2'])
            print('rocperiod = %.4f' % optimal_pars['p3'])
 
        # or heatmap showing how variables i = (y axis/ p1) and j = (x axis /p2) impact strategy outcome
        if globalparams["heatmap"]:     
            pool = multiprocess.Pool(processes=7, maxtasksperchild=1)  
            noduplicates = []
            runt = 1
            for i in eval(params_comb(globalparams["params_combination"])[0][0]):
                i = int(i)
                for j in eval(params_comb(globalparams["params_combination"])[0][1]):
                    j = int(j)
                    for k in eval(params_comb(globalparams["params_combination"])[0][2]):  
                        k = int(k)
                        if runt == 1:
                            noduplicates.append([i, j, k])
                        else:
                            if [i, j, k] in noduplicates:
                                continue
                            else:
                                noduplicates.append([i, j, k])                            
                        runt += 1
                        
            def Convert(lst):
                lst2 = [["p1","p2","p3"] for x in range(len(lst))]
                res_dct = []
                for x in range(len(lst)):
                    res_dct.append(dict(zip(lst2[x],lst[x])))
                return res_dct
            
            Convert(noduplicates)
 
            results_list = pool.starmap_async(runstrat, noduplicates,chunksize=1)
            pool.close()            
            pool.join()
            
            #sort list since multiprocessing messes up the order
            results_list = sorted(results_list.get(), key = lambda x:(x[0], x[1], x[2]))
            
            if len(globalparams["params_combination"]) == 2:
                for i in range(len(results_list)):
                    results_list[i].pop(eval(params_comb(globalparams["params_combination"])[1][1]))
                my_heatmap1(results_list)
                for i in range(len(results_list)):
                    results_list[i].pop(2)
                my_heatmap1(results_list)
            else:
                plotly_4d(results_list)
            return
 
        # add strategy
        if globalparams["optunity"]: #Pick other than Buy and Hold Strategy for optunity optimziation
            cerebro.addstrategy(eval(gmatplotliblobalparams["strategy"]), selcperc=optimal_pars['p1'], mperiod=int(optimal_pars['p2']), rocperiod=int(optimal_pars['p3']))
        else:    
            cerebro.addstrategy(eval(globalparams["strategy"]))
 
        # set the cash, cheat on close and commission
        cerebro.broker.setcash(globalparams["cash"])
        cerebro.broker.set_coc(True)
        cerebro.broker.setcommission(commission=globalparams["commission"])
 
        # Adding Analysers
        cerebro.addanalyzer(btanal.PyFolio)                # Needed to use PyFolio & Quanstat
        cerebro.addanalyzer(Sortino, _name="sortino") 
        cerebro.addobserver(bt.observers.Benchmark, data=benchdata, timeframe=bt.TimeFrame.NoTimeFrame)
 
        results = cerebro.run(maxcpus=0) #maxcpu=1 otherwise pickling multiprocessing errors
 
        # <<<Performance analysing section section>>>
        cerebro.plot()
 
        # Basic performance evaluation ... final value ... minus starting cash
        pnl = cerebro.broker.get_value() - globalparams["cash"]
        print("Sortino Ratio : ", results[0].analyzers.sortino.get_analysis()["sortino"])
        print('Profit ... or Loss: {:.2f}'.format(pnl))
 
        # Quantstats thanks to https://algotrading101.com/learn/backtrader-for-backtesting/
        # Does not work with optstrategy
        returns, positions, transactions, gross_lev = results[0].analyzers.pyfolio.get_pf_items()
        returns.index = returns.index.tz_convert(None)
        qs.reports.html(returns, output='/gdrive/MyDrive/Stock Trading/Others/stats.html', title='Momentum '+ globalparams["strategy"] + " " + str(int(globalparams["selcperc"])) + " # of stocks picked")
        webbrowser.open('/gdrive/MyDrive/Stock Trading/Others/stats.html')
 
# <<<Execute starting section>>>    
if __name__ == '__main__':
    run()

2021-04-01 Enter TSLA - Rank 4.89
2021-04-01 Enter MRNA - Rank 3.88
2021-04-06 Buy Submitted 2021-04-06 TSLA Price 691.05 Value 474751.34 Size 687 Cash 45549.17
2021-04-06 Buy Submitted 2021-04-06 MRNA Price 129.91 Value 474950.97 Size 3656 Cash 45549.17
2021-04-06 Buy 2021-04-06 TSLA Price 691.05 Value 474751.34 Size 687
2021-04-06 Buy 2021-04-06 MRNA Price 129.91 Value 474950.97 Size 3656
2021-05-06 Leave TSLA - Rank 3.32
2021-05-06 Enter AMAT - Rank 3.49
2021-05-10 Sell Submitted 2021-05-10 TSLA Price 672.37 Value -461918.19 Size -687
2021-05-10 Buy Submitted 2021-05-10 AMAT Price 132.95 Value 452162.94 Size 3401 Cash 50734.01
2021-05-10 Sell 2021-05-10 TSLA Price 672.37 Value -461918.19 Size -687
2021-05-10 Buy 2021-05-10 AMAT Price 132.95 Value 452162.94 Size 3401
2021-08-26 Leave AMAT - Rank 2.10
2021-08-26 Enter ASML - Rank 2.13
2021-08-30 Sell Submitted 2021-08-30 AMAT Price 136.55 Value -464406.56 Size -3401
2021-08-30 Buy Submitted 2021-08-30 ASML Price 832.92 Value 418958.75

<IPython.core.display.Javascript object>

Sortino Ratio :  2.2761737438698724
Profit ... or Loss: 744859.41


<IPython.core.display.Javascript object>

findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>